In [40]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate,PromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnableSequence, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


load_dotenv()

True

In [21]:
llm  = ChatOllama(model="gpt-oss:120b-cloud", temperature=0.0)

In [28]:
chat_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("Ты – опытный переводчик."),
    HumanMessagePromptTemplate.from_template("Переведи текст с английского на русский:\n\"{input_text}\"")
])

chat_template

ChatPromptTemplate(input_variables=['input_text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Ты – опытный переводчик.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input_text'], input_types={}, partial_variables={}, template='Переведи текст с английского на русский:\n"{input_text}"'), additional_kwargs={})])

In [25]:
messages = chat_template.format_messages(input_text="Hello, world!")
response = llm.invoke(messages)
response.content

'«Привет, мир!»'

In [32]:
template_str = "Привет, {% if formal %}уважаемый{% else %}дорогой{% endif %} {{ name }}!"
prompt = PromptTemplate(
    template=template_str,
    input_variables=["name", "formal"],
    template_format="jinja2"
)
print(prompt.format(name="Иван", formal=True))

Привет, уважаемый Иван!


In [36]:
template = "Отвечай как ассистент: Привет, {name}! Рад тебя видеть. Я могу помочь с вопросами по {topic}."
prompt = PromptTemplate(template=template, input_variables=["name", "topic"])

chain = prompt | llm
response = chain.invoke({"name": "Анна", "topic": "программированию"})
response.content

'Привет, Анна! 😊 Рад тебя видеть. Чем могу помочь? Если у тебя есть вопросы по программированию — от синтаксиса и отладка кода до архитектурных решений и лучших практик — просто дай знать, и мы разберём их вместе!'

In [41]:
prompt1 = PromptTemplate.from_template("Назови 5 популярных достопримечательностей в городе {city}.")
prompt2 = PromptTemplate.from_template("Составь маршрут на 3 дня по городу {city}, включив следующие места: {places_list}.")

attractions_chain = prompt1 | llm | StrOutputParser()

overall_chain = {
    "places_list": attractions_chain,
    "city": RunnablePassthrough()
} | prompt2 | llm | StrOutputParser()

result = overall_chain.invoke({"city": "Париж"})

result

"## 3‑дневный маршрут по Парижу  \n**Город:** Париж  \n**Главные точки:** Эйфелева башня, Лувр, Нотр‑Дам, Триумфальная арка, Монмартр\u202f+\u202fСакре‑Кёр  \n\n> **Полезные советы**  \n> * **Метро**\u202f– самый быстрый способ перемещения. Приобретите «Paris Visite» (1‑3\u202fдня) или «Navigo Découverte» (недельный проездной).  \n> * **Билеты онлайн**\u202f– экономят время в очередях (Эйфелева башня, Лувр, Триумфальная арка, Сакре‑Кёр).  \n> * **Одежда**\u202f– в марте‑апреле/сентябре‑октябре погода переменчивая, берите лёгкую куртку и удобную обувь.  \n> * **Питание**\u202f– в каждом районе есть проверенные кафе/ресторанчики (см. рекомендации).  \n\n---\n\n## День\u202f1 – «Эйфелева башня\u202f+\u202fСеверный берег Сены»\n\n| Время | Что делаем | Как добраться | Комментарий |\n|-------|------------|---------------|-------------|\n| **08:30** | Завтрак в кафе **Café de l’Homme** (Rive Gauche, рядом с Трокадеро) | Метро\u202f6\u202f– станция **Trocadéro** | Вид на башню уже из окна, от